In [1]:
from load_data import *
from model import *

70484it [00:03, 18332.77it/s]


maximum sentence length : 431
minimum sentence length : 4
mean sentence length : 32.8592020884
std dev sentence length : 17.3015877763
mean + 3*std_dev sentence length : 84.7639654174


maximum paragraph length : 767
minimum paragraph length : 22
mean paragraph length : 139.626156291
std dev paragraph length : 55.4859031596
mean + 3*std_dev paragraph length : 306.08386577
Loaded 2195875 pretrained embeddings.
USE CPU


In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src1 = batch[0]
        src2 = batch[1]
        trg = batch[2]
        src1 = Variable(src1.type(torch.LongTensor))
        src2 = Variable(src2.type(torch.LongTensor))
        trg = Variable(trg.type(torch.LongTensor))
        if USE_GPU:
            src1 = src1.cuda()
            src2 = src2.cuda()
            trg = trg.cuda()
        
        optimizer.zero_grad()
        
        output = model(src1,src2, trg)
        
        #trg = [batch size,sent len]
        #output = [batch size,sent len, output dim]
        
        #reshape to:
        #trg = [(sent len - 1) * batch size]
        #output = [(sent len - 1) * batch size, output dim]
        
        loss = criterion(output[:,1:,:].contiguous().view(-1, output.shape[2]), trg[:,1:].contiguous().view(-1))
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / iterator.dataset.length,optimizer

def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src1 = batch[0]
            src2 = batch[1]
            trg = batch[2]
            src1 = Variable(src1.type(torch.LongTensor))
            src2 = Variable(src2.type(torch.LongTensor))
            trg = Variable(trg.type(torch.LongTensor))
            if USE_GPU:
                src1 = src1.cuda()
                src2 = src2.cuda()
                trg = trg.cuda()

            output = model(src1,src2, trg, 0) #turn off teacher forcing

            loss = criterion(output[:,1:,:].contiguous().view(-1, output.shape[2]), trg[:,1:].contiguous().view(-1))

            epoch_loss += loss.item()
        
    return epoch_loss / iterator.dataset.length


def save(filename):
    state = {'epoch': epoch + 1, 'state_dict': model.state_dict(),
             'optimizer': optimizer.state_dict(), 'valid_loss': valid_loss}
    torch.save(state, filename)
    
WORD_SIZE = len(word_mapping)

enc1 = EncoderParagraph(WORD_SIZE,WORD_DIM,HIDDEN_SIZE,pretrained_word_embeds)
enc2 = EncoderSentence(WORD_SIZE,WORD_DIM,HIDDEN_SIZE,pretrained_word_embeds)
dec = AttnDecoderLSTM(WORD_SIZE,WORD_DIM,HIDDEN_SIZE*2,MAX_SENT_LEN,pretrained_word_embeds)
model = QuestionGeneration(enc1,enc2, dec)

if USE_GPU:
    enc1 = enc1.cuda()
    enc2 = enc2.cuda()
    dec = dec.cuda()
    model = model.cuda()
    
    
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
optimizer = optim.Adam(model_parameters)

pad_idx = word_mapping['<pad>']
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)



#### Training

best_valid_loss = float('inf')

#if not os.path.isdir(f'{SAVE_DIR}'):
#    os.makedirs(f'{SAVE_DIR}')

for epoch in range(N_EPOCHS):
    
    train_loss,opti = train(model, train_dataloader, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_dataloader, criterion)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        save('../models/checkpoint_epoch_'+str(epoch)+'_valid_loss_'+str(valid_loss)+'_'+'.pth.tar')
    
    print('Epoch [{}/{}] Train Loss: {:.4f} | Val. Loss: {:.4f}'.format(epoch+1, N_EPOCHS, train_loss,valid_loss))